In [ ]:
import random
import re
import numpy as np
import json
from pyspark import SparkContext
from collections import defaultdict
import pandas as pd

from sklearn import svm 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import scipy.sparse

from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
import string
import features.feature as ft

In [2]:
cbd = ft.ClickbaitDataset("clickbait17-train-170331/instances.jsonl", "clickbait17-train-170331/truth.jsonl")
f = ft.NGramFeature(data=cbd, vectorizer=CountVectorizer, n=1)
_x, _y = ft.FeatureBuilder(cbd).add_feature("charonegramcount", f) \
                             .build()

In [5]:
def learn(x, y):
    from sklearn import svm 
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error
    x_train, x_test, y_train, y_test = train_test_split(x, y.T, random_state=42)
    model = svm.SVR()
    model.fit(x_train, y_train)
    y_predicted = model.predict(x_test)
    return mean_squared_error(y_test, y_predicted)

In [6]:
list_of_features = [scipy.sparse.csc_matrix(_x[:, x]) for x in range(_x.shape[1])]
data = sc.parallelize(list_of_features).map(lambda a: learn(a, _y)).collect().foreach(print)

# list_of_mse = [learn(x, _y) for x in list_of_features]
# print(list_of_mse)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 545 in stage 1.0 failed 4 times, most recent failure: Lost task 545.3 in stage 1.0 (TID 774, betaweb017.medien.uni-weimar.de, executor 743): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/media/storage2/yarn/local/usercache/mike4537/appcache/application_1495374067322_1060/container_e49_1495374067322_1060_01_001016/pyspark.zip/pyspark/worker.py", line 163, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/media/storage2/yarn/local/usercache/mike4537/appcache/application_1495374067322_1060/container_e49_1495374067322_1060_01_001016/pyspark.zip/pyspark/worker.py", line 54, in read_command
    command = serializer._read_with_length(file)
  File "/media/storage2/yarn/local/usercache/mike4537/appcache/application_1495374067322_106

In [24]:
# print(type(x_train[:,0]))
# print(count_vectorizer.inverse_transform(x_train[:,0]))
# print(count_vectorizer.get_feature_names())
print(x_train[:,0].shape)
print(x_train.shape)
print(x_train.shape[1])
# mse = train_and_test(x_train[:,2], y_train, x_test[:,2], y_test)

(1844, 1)
(1844, 7405)
7405


In [28]:
list_of_features = [scipy.sparse.csc_matrix(x_train[:, x]) for x in range(x_train.shape[1])]
list_of_features2 = [x_train[:, x] for x in range(x_train.shape[1])]
# TODO convert to spark sparse matrices/vectors if it doesnt work
# pyspark.mllib.linalg.Vectors

In [30]:
print(len(list_of_features))
print(list_of_features[0].shape)
print(list_of_features[0])
print(list_of_features2[0])
# data = sc.parallelize(list_of_features).map(lambda a: learn(a, _y)).collect().foreach(print)

7405
(1844, 1)
  (97, 0)	1
  (149, 0)	1
  (168, 0)	1
  (190, 0)	1
  (239, 0)	1
  (253, 0)	1
  (264, 0)	1
  (344, 0)	1
  (348, 0)	1
  (354, 0)	1
  (363, 0)	1
  (419, 0)	1
  (475, 0)	1
  (504, 0)	1
  (557, 0)	1
  (605, 0)	1
  (672, 0)	1
  (699, 0)	1
  (785, 0)	1
  (815, 0)	1
  (824, 0)	3
  (1079, 0)	3
  (1080, 0)	1
  (1104, 0)	1
  (1132, 0)	1
  (1144, 0)	1
  (1227, 0)	1
  (1268, 0)	1
  (1428, 0)	1
  (1434, 0)	4
  (1593, 0)	1
  (1701, 0)	1
  (1760, 0)	1
  (1768, 0)	1
  (97, 0)	1
  (149, 0)	1
  (168, 0)	1
  (190, 0)	1
  (239, 0)	1
  (253, 0)	1
  (264, 0)	1
  (344, 0)	1
  (348, 0)	1
  (354, 0)	1
  (363, 0)	1
  (419, 0)	1
  (475, 0)	1
  (504, 0)	1
  (557, 0)	1
  (605, 0)	1
  (672, 0)	1
  (699, 0)	1
  (785, 0)	1
  (815, 0)	1
  (824, 0)	3
  (1079, 0)	3
  (1080, 0)	1
  (1104, 0)	1
  (1132, 0)	1
  (1144, 0)	1
  (1227, 0)	1
  (1268, 0)	1
  (1428, 0)	1
  (1434, 0)	4
  (1593, 0)	1
  (1701, 0)	1
  (1760, 0)	1
  (1768, 0)	1


In [10]:
tokens = data                                                   \
    .map( lambda document: document['text'].strip().lower())            \
    .map( lambda document: re.split("[\s;,#]", document))       \
    .map( lambda word: [x for x in word if x.isalpha()])

In [11]:
termCounts = tokens                             \
    .flatMap(lambda document: document)         \
    .map(lambda word: (word, 1))                \
    .reduceByKey( lambda x,y: x + y)            \
    .map(lambda tuple: (tuple[1], tuple[0]))    \
    .sortByKey(False)

In [12]:
# Index each one and collect them into a map
vocabulary = termCounts                         \
    .map(lambda x: x[1])                        \
    .zipWithIndex()                             \
    .collectAsMap()

In [13]:
inv_voc = {value: key for (key, value) in vocabulary.items()}

In [14]:
# Convert the given document into a vector of word counts
def document_vector(document):
    id = document[1]
    counts = defaultdict(int)
    for token in document[0]:
        if token in vocabulary:
            token_id = vocabulary[token]
            counts[token_id] += 1
    counts = sorted(counts.items())
    keys = [x[0] for x in counts]
    values = [x[1] for x in counts]
    return (id, Vectors.sparse(len(vocabulary), keys, values))

# Process all of the documents into word vectors using the 
# `document_vector` function defined previously
documents = tokens.zipWithIndex().map(document_vector).map(list)

In [15]:
# Open an output file
with open("lda_model.txt", 'w') as f:
    lda_model = LDA.train(documents, k=50, maxIterations=int(1e3))

    topic_indices = lda_model.describeTopics(maxTermsPerTopic=int(1e3))
        
    # Print topics, showing the top-weighted 10 terms for each topic
    for i in range(len(topic_indices)):
        f.write("Topic #{0}\n".format(i + 1))
        for j in range(len(topic_indices[i][0])):
            f.write("{0}\t{1}\n".format(inv_voc[topic_indices[i][0][j]] \
                .encode('utf-8'), topic_indices[i][1][j]))
            

    f.write("{0} topics distributed over {1} documents and {2} unique words\n"  \
        .format(num_topics, documents.count(), len(vocabulary)))

Py4JJavaError: An error occurred while calling o133.trainLDAModel.
: org.apache.spark.SparkException: Job 36 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:808)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:806)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:806)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:1668)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:83)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:1587)
	at org.apache.spark.SparkContext$$anonfun$stop$8.apply$mcV$sp(SparkContext.scala:1833)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1283)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1832)
	at org.apache.sp